# CharBert

In [1]:
import __init__

In [17]:
from transformers import BertForSequenceClassification, BertConfig
from models.charbert.character_bert import CharacterBertModel
from transformers import BertTokenizer
from models.charbert.utils.character_cnn import CharacterIndexer
import numpy as np

config = BertConfig.from_pretrained('bert-base-uncased', num_labels=2)
bert_word_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bertmodel = BertForSequenceClassification(config=config)

print(">>> wordpiece embeddings whith shape of:", bertmodel.bert.embeddings.word_embeddings)

character_bert_model = CharacterBertModel.from_pretrained('../pretrained-models/general_character_bert/')
bertmodel.bert = character_bert_model
char_indexer = CharacterIndexer()  # This converts each token into a list of character indices

print(">>> wordpieces are replaced with a CharacterCNN with followint architecture\n:",
      bertmodel.bert.embeddings.word_embeddings)   


def get_bert_features(text, word_by_word=False):
    F_len = lambda X:np.array([len(x) for x in X])
    if text.isspace():
        return np.zeros((1,768))[0]
    tokenized_text = bert_word_tokenizer.basic_tokenizer.tokenize(text) #this is NOT wordpiece tokenization
    input_tensor = char_indexer.as_padded_tensor([tokenized_text])  # we build a batch of only one sequence
    #input_tensor.shape >>> torch.Size([1, 8, 50]) # (batch_size, sequence_length, character_embedding_dim)
    output = bertmodel.bert(input_tensor)[0][0]
    
    if word_by_word:
        return output
    else:
        vec = output.detach().numpy()#/F_len(tokenized_text).reshape(-1,1)
        return  sum(vec)/vec.shape[0]

>>> wordpiece embeddings whith shape of: Embedding(30522, 768, padding_idx=0)
>>> wordpieces are replaced with a CharacterCNN with followint architecture
: CharacterCNN(
  (char_conv_0): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
  (char_conv_1): Conv1d(16, 32, kernel_size=(2,), stride=(1,))
  (char_conv_2): Conv1d(16, 64, kernel_size=(3,), stride=(1,))
  (char_conv_3): Conv1d(16, 128, kernel_size=(4,), stride=(1,))
  (char_conv_4): Conv1d(16, 256, kernel_size=(5,), stride=(1,))
  (char_conv_5): Conv1d(16, 512, kernel_size=(6,), stride=(1,))
  (char_conv_6): Conv1d(16, 1024, kernel_size=(7,), stride=(1,))
  (_highways): Highway(
    (_layers): ModuleList(
      (0): Linear(in_features=2048, out_features=4096, bias=True)
      (1): Linear(in_features=2048, out_features=4096, bias=True)
    )
  )
  (_projection): Linear(in_features=2048, out_features=768, bias=True)
)


In [18]:
text = "Fuck you"

vec = get_bert_features(text, word_by_word=False)

print("the shape:", vec.shape)


the shape: (768,)


In [22]:
vec.shape

(768,)